# Tool Call LLMs
Some of the more exciting use cases for RT is when you want to cede control of the decision making process to the LLM. RT has a suite of tools that you can use to make this process much simpler. Check out an example below. 

In [11]:
from typing import Dict, Any, Set, Type
from pydantic import BaseModel, Field
import textwrap
import railtracks as rt

### Making Agents that can be used as tools:


For more information on Agents as Tools, Check out LINK TO DOCS:

In [12]:
def _critic_factory(emotion: str) -> tuple[rt.ToolManifest, str]:
    """ A factory function that creates a ToolManifest and system message for a critic tool.
    """
    system_message = f"You are a {emotion} critic of the world and you should analyze the given statement and provide a {emotion} critique of it. Be very concise and to the point."
    tool_manifest = rt.ToolManifest(
        description=f"A tool used to critique a statement {emotion}ly.",
        parameters=[rt.llm.Parameter("analysis_detail", "string", "The thing you would like to analyze")]
    )
    return system_message, tool_manifest

In [13]:
harsh_critic_system_message, harsh_critic_tool_manifest = _critic_factory("harsh")
harsh_critic_llm = rt.llm.OpenAILLM("gpt-4o")

positive_critic_system_message, positive_critic_tool_manifest = _critic_factory("positive")
positive_critic_llm = rt.llm.AnthropicLLM("claude-3-5-sonnet-20240620")

deeper_meaning_critic_system_message, deeper_meaning_critic_tool_manifest = _critic_factory("deeper")
# deeper_meaning_critic_llm = rt.llm.HuggingFaceLLM("together/deepseek-ai/DeepSeek-R1")
deeper_meaning_critic_llm = rt.llm.OpenAILLM("gpt-4o")

In [14]:
HarshCritic = rt.agent_node(
    "Harsh Critic",
    system_message=harsh_critic_system_message,
    llm_model=harsh_critic_llm,
    manifest=harsh_critic_tool_manifest,
)

PositiveCritic = rt.agent_node(
    "Positive Critic",
    system_message=positive_critic_system_message,
    llm_model=positive_critic_llm,
    manifest=positive_critic_tool_manifest,
)

DeeperMeaningCritic = rt.agent_node(
    "Deeper Meaning Critic",
    system_message=deeper_meaning_critic_system_message,
    llm_model=deeper_meaning_critic_llm,
    manifest=deeper_meaning_critic_tool_manifest,
)


### Parent Tool calling LLM that has access to agents as tools

In [15]:
class CriticOutput(BaseModel):
    harsh_critic: str = Field(description="The harsh critic of the statement")
    positive_critic: str = Field(description="The positive critic of the statement")
    deeper_meaning: str = Field(description="The deeper meaning of the statement")
    overall_score: float = Field(description="The overall score of the statement on a scale of 0 to 1")

In [18]:
system_message_critic = "You are a critic of the world and you provide comprehensive critiques of the world around you. You should utilize the provided tools to collect specific critiques and structure them before completing your answer."
Critic = rt.agent_node(
    "Critic",
    tool_nodes={HarshCritic, PositiveCritic, DeeperMeaningCritic},
    output_schema=CriticOutput,
    system_message=system_message_critic,
    llm_model=rt.llm.OpenAILLM("gpt-4o")
)

C:\Users\Aryan\dev\rc\packages\railtracks\src\railtracks\validation\node_creation\validation.py:222: UserWarning: You have set max_tool_calls to None. The llm will be able to make unlimited tool calls.
  warnings.warn(


For the parent node we will dynamically inject the model during `rt.call` 

In [19]:
template = (
    "I am writing a short story and I would like to analyze my introduction.\n"
    "\n"
    "Once upon a time there was a little boy who lived in a small village. He was a very kind and generous, but lacked an understanding"
    " of the world around him. He was always looking for ways to help others and make the world a better place. One day, he stumbled upon a"
    " magical book that would change his life forever. The book was filled with stories of adventure and mystery, and the promise of a better"
    " tomorrow."
)
with rt.Session(timeout=50):
    response = await rt.call(
        Critic, template, max_tool_calls=10
    )

Using existing session


C:\Users\Aryan\dev\rc\packages\railtracks\src\railtracks\nodes\concrete\_tool_call_base.py:74: UserWarning: You have provided max_tool_calls as a parameter and as a class variable. We will use the parameter.
  warnings.warn(
[+422.366s] RT          : INFO     - START CREATED Critic
[+425.677s] RT          : INFO     - Critic CREATED Harsh Critic
[+425.683s] RT          : INFO     - Critic CREATED Deeper Meaning Critic
[+425.696s] RT          : INFO     - Critic CREATED Positive Critic
[+428.066s] RT          : INFO     - Positive Critic DONE
[+430.375s] RT          : INFO     - Harsh Critic DONE
[+432.294s] RT          : INFO     - Deeper Meaning Critic DONE
[+438.139s] RT          : INFO     - Critic CREATED Structured LLM (CriticOutput)
[+444.376s] RT          : INFO     - Structured LLM (CriticOutput) DONE
[+444.383s] RT          : INFO     - Critic DONE
[+444.386s] RT.Session  : INFO     - Saving execution info to .railtracks\c99bc3f5-0682-4dcd-b5f0-4f9709fb3fbd.json


In [20]:
final_critic = response.structured
assert isinstance(final_critic, CriticOutput)
print(final_critic.model_dump_json(indent=2))

{
  "harsh_critic": "This statement is painfully trite and uninspired. It's as if it ticked every cliché in the book without bringing anything new or interesting to the table. The mention of a \"magical book\" screams of unoriginality, heavily borrowing from countless overused tropes with no effort to innovate. Moreover, the character description of a \"kind and generous\" boy lacking understanding is so one-dimensional it could put you to sleep. This narrative seems more like a lazy attempt to recycle old fairy tale elements rather than an engaging piece of storytelling. Overall, it severely lacks depth, creativity, and any semblance of originality.",
  "positive_critic": "This statement paints a charming picture of innocence and hope. It effectively introduces a protagonist with admirable qualities - kindness and generosity - while also establishing room for growth through his lack of worldly understanding. The introduction of the magical book creates intrigue and sets the stage for 

Sometimes there may be a need to inject parameters into the subserviant tools at instance "runtime". This can be done by modifying the node creation method in the parent top level node


In [ ]:
class GradeLevelHarshCritic(HarshCritic):
    def __init__(self, message_history: rt.llm.MessageHistory, grade_level: int):
        super().__init__(message_history=message_history)
        self.grade_level = grade_level

    @classmethod
    def prepare_tool(cls, tool_parameters):
        message_hist = rt.llm.MessageHistory(
            [rt.llm.UserMessage(tool_parameters["analysis_detail"])]
        )
        return cls(message_hist, tool_parameters["grade_level"])

    def system_message(self):
        return (
            super().system_message()
            + " You should provide a critique at a grade level of "
            + str(self.grade_level)
        )

In [ ]:
class InjectGradeLevel(Critic):
    def __init__(self, message_history: rt.llm.MessageHistory, grade_level: int):
        super().__init__(message_history=message_history)
        self.grade_level = grade_level

    def create_node(self, tool_name: str, arguments: Dict[str, Any]) -> rt.Node:
        if tool_name == "Harsh_Critic":
            arguments["grade_level"] = self.grade_level
            return GradeLevelHarshCritic.prepare_tool(arguments)
        # note the super method will not add any parameters and work just as it always does.
        return super().create_node(tool_name, arguments)

    def connected_nodes(self) -> Set[Type[rt.Node]]:
        # note that you can add the new node to the connected nodes set
        return {GradeLevelHarshCritic, PositiveCritic, DeeperMeaningCritic}

In [ ]:
template = (
    "I am writing a short story and I would like to analyze my introduction.\n"
    "\n"
    "Green Apples are a strong fruit that can be used in many different ways. They are a great source of vitamins and minerals, and can be eaten or even thrown at others you don't like"
)
# with rt.Runner(executor_config=rt.run.ExecutorConfig(logging_setting="QUIET", timeout=50)) as runner:
#     response = await runner.run(InjectGradeLevel,
#         message_history=rt.llm.MessageHistory([rt.llm.UserMessage(template)]),
#         grade_level=12)

rt.call(
    InjectGradeLevel,
    message_history=rt.llm.MessageHistory([rt.llm.UserMessage(template)]),
    grade_level=12,
)


print(response.answer)